In [17]:
#%run ./lib.py

In [18]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType, DecimalType, TimestampType, DataType, IntegerType
from pyspark.sql.functions import col, lit, trim, substring, concat, udf, upper, initcap
from datetime import datetime
import os

In [19]:
conf = SparkConf() \
    .setAppName("projeto_pbi") \
    .setSparkHome('./spark/home')

LAKE_HOME = os.getenv("LAKE_HOME", "/spark/home")

sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

In [20]:
TABLE_NAME = 'fact_order_payment'

def create_sk(spark, df, key_column_name, table_name):
    sk = {}
    sk = df.select(col(key_column_name).alias("key")).rdd.zipWithIndex()
    new_sk = sk.map(lambda row: list(row[0]) + [row[1] + 1])
    new_sk_map = new_sk.collectAsMap()

    sk_schema = \
        StructType(
            [StructField('key', StringType(), True),
            StructField('SK', LongType(), True)]
        )

    sk_frame = spark.createDataFrame(new_sk, sk_schema)
    sk_frame.write.mode('overwrite').csv('{}/dataset/e-commerce/02_surrogate_key/sk_{}.csv'.format(LAKE_HOME, table_name), header=True)

    return new_sk_map

def locate_sk(mapping: dict):
    return udf(lambda x: mapping.get(x), IntegerType())


def map_sk(spark, table_name):
    sk_table = spark.read.csv('{}/dataset/e-commerce/02_surrogate_key/sk_{}.csv'.format(LAKE_HOME, table_name), header=True)
    dict_sk = sk_table.rdd.map(lambda x: (x[0], int(x[1]))).collectAsMap()
    return dict_sk

In [21]:
df = spark.read.csv('{}/dataset/e-commerce/01_extract/order_payments_dataset.csv'.format(LAKE_HOME), header=True)

df_payment = \
    df.select(
        col('order_id').cast(StringType()).alias('ID_ORDER'),
        col('payment_type').cast(StringType()).alias('DES_PAYMENT_TYPE'),
        col('payment_installments').cast(IntegerType()).alias('QTT_INSTALLMENT'),
        col('payment_value').cast(DecimalType(15,2)).alias('VAL_PAYMENT'),
    )

order_sk = map_sk(spark=spark, table_name='fact_order')

In [22]:
fact_order_payment = \
    df_payment \
        .withColumn('SK_ORDER', locate_sk(order_sk)(col('ID_ORDER')))

In [23]:
fact_order_payment = \
    fact_order_payment.select(
            col('SK_ORDER'),
            col('DES_PAYMENT_TYPE'),
            col('QTT_INSTALLMENT'),
            col('VAL_PAYMENT'),  
        )

fact_order_payment.write.mode('overwrite').csv('{}/dataset/e-commerce/04_fact/{}.csv'.format(LAKE_HOME, TABLE_NAME), header=True)